## 1、快递费

#### 1.0 最终和供应商确认版账单:文件|路径

<font color = "red"/> 待定</font>
```


class YaAo():
    def __init__(self,input_path,input_file):
        self.input_path = input_path 
        self.input_file = input_file 
    
    def first_read(self,Tier,Vendor):
        self.df = pd.read_excel(os.path.join(input_path,input_file),sheet_name="2.1 快递费",header=1)
        self.df["重量段"] = df["总重量 (kg)"].map(Tier)
        self.df["物流公司"] = df["物流公司"].map(Vendor)
        self.df_pivot = df.pivot_table(index=["月份","店铺/货主","仓库区域","省份","重量段","物流公司"],
                          values=["物流单号/运单号","原始单号/外部订单号","计费重量","总重量 (kg)","快递费(总)","货品数量/拣货数量"],
                          aggfunc={"物流单号/运单号":"count","原始单号/外部订单号":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","快递费(总)":"sum",
                                  '货品数量/拣货数量':"sum"})
        self.df_pivot.rename(columns={"原始单号/外部订单号":"订单量","物流单号/运单号":"包裹量"},inplace=True)
        self.df_pivot = df_pivot.reset_index()
        
        return df_pivot
    def second_to_excel(self,output):
        self.to_excel(os.path.join(output,input_file))
        
```

In [25]:
input_path = r"D:\我的坚果云\Work  Share\许程晨\4、In Process\2、外仓供应商账单\Final"
input_file = r"【魔根001_代发仓】物流费用账单 202204 v1.0.xlsx"

#### 1.1 读入文件

In [26]:
import os
import pandas as pd
import math

In [27]:
df = pd.read_excel(os.path.join(input_path,input_file),sheet_name="4.1 【B2C】订单处理费、快递费、快递包材费",header=0)
df[:5]

,运输类型,仓库区域,仓库名称,店铺/货主,订单类型,订单日期,发货日期,订单编号/包裹号,出库单号/销售订单编码,物流单号/运单号,...,省份.1,城市.1,地区,称重重量,货品数量.1,运费,深圳加收,包材费.1,订单处理费.1,疫情加收
0,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 22:12:36,2022-04-27 13:38:41,JY202204272455,CK202204276717,75874043750029,...,广东省,湛江市,广东省 湛江市 徐闻县,2.88,3,3.85,NaN,1.84,0.60,NaN
1,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 12:35:45,2022-04-27 13:38:41,JY202204272456,CK202204276716,75874043750031,...,山东省,青岛市,山东省 青岛市 城阳区,2.88,3,3.85,NaN,1.84,0.60,0.2
2,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-15 14:56:45,2022-04-27 11:03:39,JY202204154890,CK2022042642118,75873992751978,...,辽宁省,沈阳市,辽宁省 沈阳市 和平区,1.92,2,3.20,NaN,1.47,0.55,NaN
3,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 13:08:34,2022-04-26 20:30:20,JY202204254918,CK2022042642116,75873857399112,...,北京,北京市,北京 北京市 海淀区,1.92,2,4.20,NaN,1.47,0.55,NaN
4,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 15:34:38,2022-04-26 20:30:20,JY202204257148,CK2022042642115,75873857399125,...,北京,北京市,北京 北京市 海淀区,1.92,2,4.20,NaN,1.47,0.55,NaN


In [28]:
df.columns

Index(['运输类型', '仓库区域', '仓库名称', '店铺/货主', '订单类型', '订单日期', '发货日期', '订单编号/包裹号',
       '出库单号/销售订单编码', '物流单号/运单号', '原始单号/外部订单号', '货品数量', '总体积 (m³)', '总重量 (kg)',
       '客户网名', '客服备注', '物流公司', '省份', '城市', '区县', '包材编码', '包装耗材', '包装数量',
       '包装单价', '包材费', '账单类型', '重量段 (订单处理)', '首件 (元)', '续件 (数量)', '订单处理费',
       '重量段 (快递费)', '首重 (kg)', '续重 (kg)', '首重 (元)', '续重 (元/kg)', '深圳加收(元/票)',
       '快递费', '快递费(002)', 'Unnamed: 38', '原始单号', '发货时间', '店铺名称', '货主',
       '物流公司.1', '物流单号', '省份.1', '城市.1', '地区', '称重重量', '货品数量.1', '运费', '深圳加收',
       '包材费.1', '订单处理费.1', '疫情加收'],
      dtype='object')

#### 1.2 定义重量段

In [29]:
def Tier(weight):
    if weight > 10 :
        Tier = "X >10kg"
    elif weight > 5 :
        Tier = "X >5kg,≤10kg"
    elif weight > 3 :
        Tier = "X >3kg,≤5kg"
    elif weight > 2 :
        Tier = "X >2kg, ≤3kg"
    elif weight > 1 :
        Tier = "X >1kg, ≤2kg"
    elif weight > 0.5 :
        Tier = "X >0.5kg, ≤1kg"
    elif weight > 0.3 :
        Tier = "X >0.3kg, ≤0.5kg"
    else :
        Tier = "X ≤0.3kg"
    return Tier 

In [30]:
df["重量段"] = df["总重量 (kg)"].map(Tier)
df["计费重量"] = df["总重量 (kg)"].map(math.ceil)
df["订单量"] = 1
df["订单量"][df["原始单号/外部订单号"].duplicated()==True] = 0

C:\Users\CHENGC~1\AppData\Local\Temp/ipykernel_204/951243271.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["订单量"][df["原始单号/外部订单号"].duplicated()==True] = 0


In [31]:
df["原始单号/外部订单号"].duplicated().value_counts()

False    35530
True        58
Name: 原始单号/外部订单号, dtype: int64

#### 1.3 定义物流公司

<font color="red"> 待定 </font>
```
def Vendor(x):
    if x == "中通标快（普冷）":
        Vendor = "中通快递"
    elif x == "顺丰标快（普冷）":
        Vendor = "顺丰快递"
    return Vendor 

df["物流公司"] = df["物流公司"].map(Vendor)
```

#### 1.4 Pivot

In [32]:
df_pivot = df.pivot_table(index=["店铺/货主","仓库区域","省份","重量段","物流公司"],
                          values=["物流单号/运单号","订单量","计费重量","总重量 (kg)","快递费(002)","货品数量"],
                          aggfunc={"物流单号/运单号":"count","订单量":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","快递费(002)":"sum",
                                  '货品数量':"sum"},margins=True)

df_pivot

快递费(002)  总重量 (kg)  物流单号/运单号  \
店铺/货主        仓库区域 省份   重量段            物流公司                                  
抖音小店-那街那巷旗舰店 上海   北京   X >0.5kg, ≤1kg 中通快递      31.20      7.68         8   
                       X >1kg, ≤2kg   中通快递       8.40      3.84         2   
                  安徽省  X >0.5kg, ≤1kg 中通快递       2.90      0.96         1   
                  山东省  X >2kg, ≤3kg   中通快递       4.15      2.88         1   
                  山西省  X >2kg, ≤3kg   中通快递       3.85      2.88         1   
...                                               ...       ...       ...   
那街那巷旗舰店      上海   黑龙江省 X >1kg, ≤2kg   圆通快递      16.30      9.60         5   
                                      顺丰快递      23.00      1.92         1   
                       X >2kg, ≤3kg   圆通快递       7.70      5.76         2   
                       X >3kg,≤5kg    顺丰快递      42.00      3.85         1   
All                                         136815.85  36832.43     35588   

                                             计费重量    订单量   货品数量  
店铺/货主        仓库区域 省份   重量段            物流公司                       
抖音小店-那街那巷旗舰店 上海   北京   X >0.5kg, ≤1kg 中通快递      8      8      8  
                       X >1kg, ≤2kg   中通快递      4      2      4  
                  安徽省  X >0.5kg, ≤1kg 中通快递      1      1      1  
                  山东省  X >2kg, ≤3kg   中通快递      3      1      3  
                  山西省  X >2kg, ≤3kg   中通快递      3      1      3  
...                                           ...    ...    ...  
那街那巷旗舰店      上海   黑龙江省 X >1kg, ≤2kg   圆通快递     10      4     10  
                                      顺丰快递      2      1      2  
                       X >2kg, ≤3kg   圆通快递      6      2      6  
                       X >3kg,≤5kg    顺丰快递      4      1      4  
All                                         38354  35530  38084  

[252 rows x 6 columns]

In [33]:
df_pivot.rename(columns={"物流单号/运单号":"包裹量"},inplace=True)

In [34]:
df_pivot = df_pivot.reset_index()

df_pivot

,店铺/货主,仓库区域,省份,重量段,物流公司,快递费(002),总重量 (kg),包裹量,计费重量,订单量,货品数量
0,抖音小店-那街那巷旗舰店,上海,北京,"X >0.5kg, ≤1kg",中通快递,31.20,7.68,8,8,8,8
1,抖音小店-那街那巷旗舰店,上海,北京,"X >1kg, ≤2kg",中通快递,8.40,3.84,2,4,2,4
2,抖音小店-那街那巷旗舰店,上海,安徽省,"X >0.5kg, ≤1kg",中通快递,2.90,0.96,1,1,1,1
3,抖音小店-那街那巷旗舰店,上海,山东省,"X >2kg, ≤3kg",中通快递,4.15,2.88,1,3,1,3
4,抖音小店-那街那巷旗舰店,上海,山西省,"X >2kg, ≤3kg",中通快递,3.85,2.88,1,3,1,3
...,...,...,...,...,...,...,...,...,...,...,...
247,那街那巷旗舰店,上海,黑龙江省,"X >1kg, ≤2kg",圆通快递,16.30,9.60,5,10,4,10
248,那街那巷旗舰店,上海,黑龙江省,"X >1kg, ≤2kg",顺丰快递,23.00,1.92,1,2,1,2
249,那街那巷旗舰店,上海,黑龙江省,"X >2kg, ≤3kg",圆通快递,7.70,5.76,2,6,2,6
250,那街那巷旗舰店,上海,黑龙江省,"X >3kg,≤5kg",顺丰快递,42.00,3.85,1,4,1,4


#### 1.5 输出pivot

In [35]:
output = r"D:\我的坚果云\Work  Share\许程晨\5、Billing Summary\8、VLC_2022"

In [36]:
df_pivot.to_excel(os.path.join(output,input_file))

## 2、订单处理费

In [37]:
df

,运输类型,仓库区域,仓库名称,店铺/货主,订单类型,订单日期,发货日期,订单编号/包裹号,出库单号/销售订单编码,物流单号/运单号,...,称重重量,货品数量.1,运费,深圳加收,包材费.1,订单处理费.1,疫情加收,重量段,计费重量,订单量
0,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 22:12:36,2022-04-27 13:38:41,JY202204272455,CK202204276717,75874043750029,...,2.88,3,3.85,NaN,1.84,0.60,NaN,"X >2kg, ≤3kg",3,1
1,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 12:35:45,2022-04-27 13:38:41,JY202204272456,CK202204276716,75874043750031,...,2.88,3,3.85,NaN,1.84,0.60,0.2,"X >2kg, ≤3kg",3,1
2,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-15 14:56:45,2022-04-27 11:03:39,JY202204154890,CK2022042642118,75873992751978,...,1.92,2,3.20,NaN,1.47,0.55,NaN,"X >1kg, ≤2kg",2,1
3,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 13:08:34,2022-04-26 20:30:20,JY202204254918,CK2022042642116,75873857399112,...,1.92,2,4.20,NaN,1.47,0.55,NaN,"X >1kg, ≤2kg",2,1
4,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 15:34:38,2022-04-26 20:30:20,JY202204257148,CK2022042642115,75873857399125,...,1.92,2,4.20,NaN,1.47,0.55,NaN,"X >1kg, ≤2kg",2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35583,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 23:41:21,2022-04-26 10:21:31,JY2022042548430,CK20220426725,9887339366021,...,0.96,1,2.90,NaN,0.95,0.50,NaN,"X >0.5kg, ≤1kg",1,1
35584,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-26 00:00:52,2022-04-26 09:11:58,JY202204260109,CK20220426575,9887337315184,...,0.96,1,2.90,NaN,0.95,0.50,NaN,"X >0.5kg, ≤1kg",1,1
35585,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-25 23:52:47,2022-04-26 09:11:58,JY2022042549025,CK20220426335,9887337387065,...,0.96,1,2.90,NaN,0.95,0.50,NaN,"X >0.5kg, ≤1kg",1,1
35586,B2C-正向 (常规),上海,魔根001_代发仓,那街那巷旗舰店,销售订单,2022-04-15 13:15:45,2022-04-16 08:57:24,JY202204153945,CK202204157554,YT6426266690922,...,0.96,1,2.90,NaN,0.95,0.50,NaN,"X >0.5kg, ≤1kg",1,1


In [38]:
df_pivot_order_handing = df.pivot_table(index=["店铺/货主","仓库区域","重量段"],
                          values=["物流单号/运单号","订单量","计费重量","总重量 (kg)","订单处理费","货品数量"],
                          aggfunc={"物流单号/运单号":"count","订单量":"sum",
                                   "计费重量":"sum","总重量 (kg)":"sum","订单处理费":"sum",
                                  '货品数量':"sum"},margins=True)

df_pivot_order_handing
df_pivot_order_handing.rename(columns={"物流单号/运单号":"包裹量"},inplace=True)
df_pivot_order_handing = df_pivot_order_handing.reset_index()

df_pivot_order_handing

,店铺/货主,仓库区域,重量段,总重量 (kg),包裹量,计费重量,订单处理费,订单量,货品数量
0,抖音小店-那街那巷旗舰店,上海,"X >0.5kg, ≤1kg",22.08,23,23,11.50,23,23
1,抖音小店-那街那巷旗舰店,上海,"X >1kg, ≤2kg",15.36,8,16,4.40,8,16
2,抖音小店-那街那巷旗舰店,上海,"X >2kg, ≤3kg",5.76,2,6,1.20,2,6
3,那街那巷旗舰店,上海,"X >0.5kg, ≤1kg",31988.16,33321,33321,16660.90,33317,33329
4,那街那巷旗舰店,上海,"X >1kg, ≤2kg",3599.80,1869,3738,1014.03,1838,3460
5,那街那巷旗舰店,上海,"X >2kg, ≤3kg",622.08,216,648,129.60,195,648
6,那街那巷旗舰店,上海,"X >3kg,≤5kg",573.59,148,596,96.72,146,596
7,那街那巷旗舰店,上海,"X >5kg,≤10kg",5.60,1,6,1.19,1,6
8,All,,,36832.43,35588,38354,17919.54,35530,38084


In [39]:
df_pivot_order_handing.to_excel(os.path.join(output,"order"+input_file))